In [3]:
## Written by: Wai Zin Linn
## Attribution: solutions to CS540 P5 2021 from Hugh Liu and Hongtao Hao
# Jayden Ye, Joshua Dietrich, and Xi Zhang

import numpy as np
import math

# modify weight and height by yourself
width, height = 55, 55
center_idx = int((width-1)/2)

M = np.zeros([height*2+1, width*3+1]) # space

# example-maze is the example maze shown to you. You can copy and past 
# the content in the example maze into a txt file and name it `example-maze.txt`

file = open('example-maze.txt', 'r')
data = []
for row in file:
    data.append(row.strip())

for h in range(height*2+1):
    for w in range(width*3+1):
        if data[h][w] == ' ':
            M[h,w] = 0 # 0 for ' '
        if data[h][w] == '+':
            M[h,w] = 1 # 1 for "+"
        if data[h][w] == '-':
            M[h,w] = 2 # 2 for '-'
        if data[h][w] == '|':
            M[h,w] = 3 # 3 for '|'

class Cell:
    def __init__(self, i, j):
        self.i = i
        self.j = j
        self.succ = ''
        self.action = ''  # which action the parent takes to get this cell
cells = [[Cell(i,j) for j in range(width)] for i in range(height)]

succ_matrix = []
for i in range(1,len(data),2):
    curr_row = []
    for j in range(1,len(data[0])-1,3):
        curr_cell = ''
        if data[i-1][j] == ' ':
            if i != 1: # prevent leaving the maze
                curr_cell += 'U'
        if data[i+1][j] == ' ':
            if i != len(data)-2: # prevent leaving the maze
                curr_cell += 'D'
        if data[i][j-1] == ' ':
            curr_cell += 'L'
        if data[i][j+2] == ' ':
            curr_cell += 'R'
        curr_row.append(curr_cell)
    succ_matrix.append(curr_row)

for i in range(height):
    for j in range(width):
        cells[i][j].succ = succ_matrix[i][j]

## SUCCESSOR MATRIX
## q2
with open('q2.txt', 'w') as f:
    f.write("\n".join([",".join([cell_col.succ for cell_col in cell_row]) for cell_row in cells]))

# bfs
visited = set()
# entrance:
s1 = {(0,center_idx)}
s2 = set()
while (height - 1, center_idx) not in visited:
    for a in s1:
        visited.add(a)
        i, j = a[0], a[1]
        succ = cells[i][j].succ
        if 'U' in succ and (i-1,j) not in (s1 | s2 | visited): 
            s2.add((i-1,j))
            cells[i-1][j].action = 'U'
        if 'D' in succ and (i+1,j) not in (s1 | s2 | visited): 
            s2.add((i+1,j))
            cells[i+1][j].action = 'D'
        if 'L' in succ and (i,j-1) not in (s1 | s2 | visited): 
            s2.add((i,j-1))
            cells[i][j-1].action = 'L'
        if 'R' in succ and (i,j+1) not in (s1 | s2 | visited): 
            s2.add((i,j+1))
            cells[i][j+1].action = 'R'     
    s1 = s2
    s2 = set()

# TODO: Modify the codes below to print 
# the list of squares searched by BFS 
## q5
with open('q5.txt', 'w') as f:
    for h in range(height):
        for w in range(width):
            f.write("1" if (h, w) in visited else "0")
            if w != width - 1:
                f.write(",")
        f.write("\n")

cur = (height - 1, center_idx)
s = ''
seq = []
while cur != (0, center_idx):
    seq.append(cur)
    i, j = cur[0], cur[1]
    t = cells[i][j].action
    s += t
    if t == 'U': cur = (i+1, j)
    if t == 'D': cur = (i-1, j)
    if t == 'L': cur = (i, j+1)
    if t == 'R': cur = (i, j-1)
action = s[::-1] # reverse

# action sequence
## q3 
with open('q3.txt', 'w') as f:
    f.write(action)

seq.append((0, center_idx))
seq = seq[::-1]

for (a,b) in seq:
    M[2*a+1, 3*b+1] = 4
    M[2*a+1, 3*b+2] = 4
    if (a+1,b) in seq and M[2*a+2, 3*b+1] != 2:
        M[2*a+2, 3*b+1] = 4
        M[2*a+2, 3*b+2] = 4

    if (a,b-1) in seq and M[2*a+1, 3*b] != 1 and M[2*a+1, 3*b] != 3:
        M[2*a+1, 3*b] = 4

M[0,3*center_idx+1] = 4
M[0,3*center_idx+2] = 4

M[2*height,3*center_idx+1] = 4
M[2*height,3*center_idx+2] = 4

# MAZE WITH SOLUTION: 
## q4
with open('q4.txt', 'w') as f:
    for h in range(height*2+1):
        for w in range(width*3+1):
            if M[h,w]==0:
                f.write(' ')
            if M[h,w]==1:
                f.write('+')
            if M[h,w]==2:
                f.write('-')
            if M[h,w]==3:
                f.write('|')
            if M[h,w]==4:
                f.write('@')
        f.write("\n")

# dfs
visited = set()
s1 = [(0, center_idx)]
s2 = set()
while (height-1, center_idx) not in visited:
    for a in s1:
        visited.add(a)
        i, j = a[0], a[1]
        succ = cells[i][j].succ
        if 'U' in succ and (i - 1, j) not in (s2 | visited) and (i - 1, j) not in s1:
            s2.add((i - 1, j))
            cells[i - 1][j].action = 'U'
        if 'D' in succ and (i + 1, j) not in (s2 | visited) and (i + 1, j) not in s1:
            s2.add((i + 1, j))
            cells[i + 1][j].action = 'D'
        if 'L' in succ and (i, j - 1) not in (s2 | visited) and (i, j - 1) not in s1:
            s2.add((i, j - 1))
            cells[i][j - 1].action = 'L'
        if 'R' in succ and (i, j + 1) not in (s2 | visited) and (i, j + 1) not in s1:
            s2.add((i, j + 1))
            cells[i][j + 1].action = 'R'
    for b in s2:
        s1.append(b)
    s1.reverse()
    s2 = set()

# DFS visited
##q6
with open("q6.txt", "w") as f:
    for h in range(height):
        for w in range(width):
            f.write("1" if (h, w) in visited else "0")
            if w != width - 1:
                f.write(",")
        f.write("\n")

## Part2
man = {(i,j): abs(i-(height - 1)) + abs(j-center_idx) for j in range(width) for i in range(height)}
euc = {(i,j): math.sqrt((i-(height-1))**2 + (j-center_idx)**2 ) for j in range(width) for i in range(height)}

# TODO: modify codes below to print the manhatten distances to the goal for each square
##q7
with open('q7.txt', 'w') as f:
    for h in range(height):
        for w in range(width):
            f.write(str(man[(h,w)]) + ",")
            if w != width - 1:
                f.write(",")
        f.write("\n")

def a_star_search(height, width, dist_method, man, euc):
    g = {(i,j): float('inf') for j in range(width) for i in range(height)}
    g[(0, center_idx)] = 0

    queue = [(0,center_idx)]
    visited = set()

    while queue and (height - 1,center_idx) not in visited:
        if dist_method == 'manhattan':
            queue.sort(key=lambda x: g[x] + man[x])
        elif dist_method == 'euclidean':
            queue.sort(key=lambda x: g[x] + euc[x])
        else:
            print('distance method should be either mahattan or euclidean!')
        point = queue.pop(0)
        if point not in visited:
            visited.add(point)
            i, j = point[0], point[1]
            succ = cells[i][j].succ
            if 'U' in succ and (i-1,j) not in visited:
                if (i-1,j) not in queue: queue += [(i-1,j)]
                g[(i-1,j)] = min(g[(i-1,j)], g[(i,j)]+1)
            if 'D' in succ and (i+1,j) not in visited:
                if (i+1,j) not in queue: queue += [(i+1,j)]
                g[(i+1,j)] = min(g[(i+1,j)], g[(i,j)]+1)
            if 'L' in succ and (i,j-1) not in visited:
                if (i,j-1) not in queue: queue += [(i,j-1)]
                g[(i,j-1)] = min(g[(i,j-1)], g[(i,j)]+1)
            if 'R' in succ and (i,j+1) not in visited:
                if (i,j+1) not in queue: queue += [(i,j+1)]
                g[(i,j+1)] = min(g[(i,j+1)], g[(i,j)]+1) 
    return visited

# list of squares searched by A* with Manhattan distance to the goal as the heuristic
a_star_man_visited = a_star_search(height, width, 'manhattan', man, euc)

# list of squares searched by A* with Euclidean distance to the goal as the heuristic
a_star_euclidean_visited = a_star_search(height, width, 'euclidean', man, euc)     

# Modify the codes below for Q8 and Q9
##q8
with open('q8.txt', 'w') as f:
    for h in range(height):
        for w in range(width):
            f.write("1," if (h, w) in a_star_man_visited else "0,")
        f.write("\n")

##q9
with open('q9.txt', 'w') as f:
    for h in range(height):
        for w in range(width):
            f.write("1," if (h, w) in a_star_euclidean_visited else "0,")
        f.write("\n")